# Spark in Colab

In [1]:
!apt-get update
!wget --no-check-certificate https://archive.apache.org/dist/spark/spark-2.2.1/spark-2.2.1-bin-hadoop2.7.tgz -O spark-2.2.1-bin-hadoop2.7.tar.gz
!gunzip spark-2.2.1-bin-hadoop2.7.tar.gz
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!tar xf spark-2.2.1-bin-hadoop2.7.tar


Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [70.5 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 k

## Loading libraries

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.2.1-bin-hadoop2.7"

In [7]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.2.1-bin-hadoop2.7"
from google.colab import drive
drive.mount('/content/drive')
import random
import re
!pip install findspark
import findspark
findspark.init()
import numpy as np
import pandas as pd
from datetime import date, datetime
from collections import Counter
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col, when
import pyspark.sql.functions as f

spark = SparkSession.builder.master("local[*]").getOrCreate()

import pyspark.sql.functions as F
from pyspark.sql.types import StringType, DoubleType, IntegerType, ArrayType, DateType

from pyspark.mllib.stat import Statistics
from pyspark.mllib.evaluation import MulticlassMetrics

from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, MinMaxScaler
from pyspark.ml.stat import Correlation, ChiSquareTest
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Loading data to design new functions

In [0]:
DATAPATH = os.path.join('content', 'drive', 'My Drive', 'Data') 
ruta_datos = "/" + DATAPATH + "/bank_uci/bank-full.csv"

In [12]:
auto = spark.read.csv(ruta_datos, sep=';', header=True, inferSchema=True)
auto.show()

+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
|age|         job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
| 58|  management| married| tertiary|     no|   2143|    yes|  no|unknown|  5|  may|     261|       1|   -1|       0| unknown| no|
| 44|  technician|  single|secondary|     no|     29|    yes|  no|unknown|  5|  may|     151|       1|   -1|       0| unknown| no|
| 33|entrepreneur| married|secondary|     no|      2|    yes| yes|unknown|  5|  may|      76|       1|   -1|       0| unknown| no|
| 47| blue-collar| married|  unknown|     no|   1506|    yes|  no|unknown|  5|  may|      92|       1|   -1|       0| unknown| no|
| 33|     unknown|  single|  unknown|     no|      1|     no|  no|unknown|  5|  may

In [16]:
from random import randint

auto = auto.withColumn("nulltest", lit(None).cast(StringType()))
auto = auto.withColumn("nulltest", when(
        col("job") == "management",
        1
    ).otherwise(col("nulltest")))
auto = auto.withColumn("nulltest", when(
        col("job") == "technician",
        2
    ).otherwise(col("nulltest")))
auto = auto.withColumn("nulltest", when(
        col("job") == "entrepreneur",
        3
    ).otherwise(col("nulltest")))
auto.show()

+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+--------+
|age|         job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|nulltest|
+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+--------+
| 58|  management| married| tertiary|     no|   2143|    yes|  no|unknown|  5|  may|     261|       1|   -1|       0| unknown| no|       1|
| 44|  technician|  single|secondary|     no|     29|    yes|  no|unknown|  5|  may|     151|       1|   -1|       0| unknown| no|       2|
| 33|entrepreneur| married|secondary|     no|      2|    yes| yes|unknown|  5|  may|      76|       1|   -1|       0| unknown| no|       3|
| 47| blue-collar| married|  unknown|     no|   1506|    yes|  no|unknown|  5|  may|      92|       1|   -1|       0| unknown| no|    null|
| 33|     unknown|  

In [20]:
## Custom made function to fill NAs with mean, minimum, maximum or median for a given set of columns

from pyspark.sql import DataFrameStatFunctions as stat

def multiple_fill_na(df, include=set(), method=set()): 
      valid = {"mean" , "minimum", "maximum", "median"}
      if method not in valid:
          raise ValueError("results: status must be one of %r." % valid)
      elif method == "mean":
        #compute the mean of the given column(s) and store it.
        stats = df.agg(*(
          F.mean(c).alias(c) for c in df.columns if c in include
        ))
        dicti = stats.first().asDict()

      elif method == "minimum":
        #compute the minimum of the given column(s) and store it.
        stats = df.agg(*(
          F.min(c).alias(c) for c in df.columns if c in include
        ))
        dicti = stats.first().asDict()

      elif method == "maximum":
        #compute the maximum of the given column(s) and store it.
        stats = df.agg(*(
          F.max(c).alias(c) for c in df.columns if c in include
        ))
        dicti = stats.first().asDict()
      elif method == "median":
          ## filter numeric cols
          num_cols = [col_type[0] for col_type in filter(lambda dtype: dtype[1] in {"bigint", "double", "int"}, 
          df.dtypes)]
          ### Compute a dict with <col_name, median_value>
          dicti = dict()
          for c in num_cols:
            dicti[c] = df.stat.approxQuantile(c, [0.5], 0.001)[0] 
      return df.na.fill(dicti)


auto_modified = multiple_fill_na(auto, "nulltest", "mean")
auto_modified.show()

+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+------------------+
|age|         job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|          nulltest|
+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+------------------+
| 58|  management| married| tertiary|     no|   2143|    yes|  no|unknown|  5|  may|     261|       1|   -1|       0| unknown| no|                 1|
| 44|  technician|  single|secondary|     no|     29|    yes|  no|unknown|  5|  may|     151|       1|   -1|       0| unknown| no|                 2|
| 33|entrepreneur| married|secondary|     no|      2|    yes| yes|unknown|  5|  may|      76|       1|   -1|       0| unknown| no|                 3|
| 47| blue-collar| married|  unknown|     no|   1506|    yes|  no|unknown|  5|  may|      92|       

In [0]:
fill_values = {column: df_data.agg({column:"mean"}).flatMap(list)[0] for column in df_data.columns if column not in ['date','id']}
df_data = df_data.na.fill(fill_values)